# Script to convert DiMESTV TIFF to videos 

In [1]:
import os
import read

# list of files ending in .tif in 'munaretto/2022-11-21_Bortolon'
files = os.listdir('munaretto/2022-11-21_Bortolon')
processed = os.listdir('/home/maana/mp_2022_34_53/dimes')
processed = [file.split('.')[0] for file in processed]
for file in files:
    if file.endswith('.tif'):
        
        shot = file.split('_')[0]
        if shot in processed:
            print(f'now reading {file} - skip video already processed')
        else:
            print(f'now reading {file}')
            read.dimes_movie(f'munaretto/2022-11-21_Bortolon/{file}')


now reading 192963_CII_1x1_1ms_low.tif
now reading 192964_CII_1x1_1ms_low.tif
now reading 192960_DaplhaND_1x1_5ms_low.tif
now reading 192970_CII_1x1_1ms_low.tif
now reading 192969_CII_1x1_1ms_low.tif
now reading 192974_CII_1x1_1ms_low.tif
now reading 192973_CII_1x1_1ms_low.tif
now reading fiducial.tif


IndexError: list index out of range